In [1]:
import time
import os
import json
import csv
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
from tqdm import tqdm_notebook

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By

In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
from collections import defaultdict


In [5]:
def get_keywordUrl(keyword):
    """ ()
    Args:
        keyword(string): 검색할 키워드
    Returns:
        url(string): keyword가 포함된 url
    """
    url = "https://search.shopping.naver.com/search/all?frm=NVSHMDL&origQuery=" + keyword +"&pagingIndex=1&pagingSize=40&productSet=model&query=" + keyword + "&sort=rel&timestamp=&viewType=list"
    return url
    
    

In [6]:
def get_detailUrl(url):
    """
    Args:
        url(string): keyword가 포함된 url
    Returns:
        url_list(list): 각 상품의 상세페이지 링크를 담은 리스트
    """
    
    options = webdriver.ChromeOptions()	
    options.add_argument('headless')	
    options.add_argument('disable-gpu')	
    driver = webdriver.Chrome('chromedriver.exe', options=options)
    driver.get(url)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)

    url_list = driver.find_elements_by_tag_name("div.basicList_title__3P9Q7 > a")
    url_list = [url.get_attribute('href') for url in url_list]
    return url_list

In [7]:
def get_prdInfo(url):
    """
    Args:
        url(string): 각 상품의 상세page 
    Returns:
        info_list(list): 상품이름, 가격, 제조사, 브랜드명, 등록일, 찜(좋아요) 갯수,
    """
    
    req = requests.get(url)
    cont = req.content
    soup = BeautifulSoup(cont, 'html.parser')
    
    prdInfo_list = list()
    
    # 제조사, 브랜드 모두 있는 경우
    try:
        # 제목
        title = soup.select('div > div.h_area > h2')
        title = title[0].get_text().replace(' ', "").replace("\n",'')
        # 평점
        try:
            score = soup.select('div > div.h_area > div > div.gpa')[0].text
        except:
            score = "NULL"
        # 제조사
        try:
            manufacter = soup.select(' div.summary_info._itemSection > div > div.goods_info > div > span:nth-of-type(1) > em')[0].text
        except:
            manufacter = "NULL"
        # 브랜드
        try: 
            brand = soup.select('div.summary_info._itemSection > div > div.goods_info > div > span:nth-of-type(2) > em')[0].text
        except:
            brand = "NULL"
        # 등록일
        try: 
            reg_date = soup.select('div.summary_info._itemSection > div > div.goods_info > div > span:nth-of-type(3) > em')[0].text
        except:
            reg_date = "NULL"
        # 좋아요 수
        try:
            num_like = soup.select('#jjim > em.cnt._keepCount')[0].text
        except:
            num_like = "NULL"
        # 가격
        try:
            price = soup.select('div.summary_cet > div.price_area > span > em')[0].text
        except:
            price = "NULL"
    except:
        # 제목
        title = soup.select('div > div.h_area > h2')
        title = title[0].get_text().replace(' ', "").replace("\n",'')
        # 평점
        try:
            score = soup.select('div > div.h_area > div > div.gpa')[0].text
        except:
            scroe = "NULL"
        # 제조사
        manufacter = "NULL"
        # 브랜드
        try:
            brand = soup.select(' div.summary_info._itemSection > div > div.goods_info > div > span:nth-of-type(1) > em')[0].text
        except:
            brand = "NULL"
        # 등록일
        try:
            reg_date = soup.select('div.summary_info._itemSection > div > div.goods_info > div > span:nth-of-type(2) > em')[0].text
        except:
            reg_date = "NULL"
        # 좋아요 수
        try:
            num_like = soup.select('#jjim > em.cnt._keepCount')[0].text
        except:
            num_like = "NULL"
        # 가격
        try: 
            price = soup.select('div.summary_cet > div.price_area > span > em')[0].text
        except:
            price = "NULL"
    prdInfo_list.append(title)
    prdInfo_list.append(score)
    prdInfo_list.append(manufacter)
    prdInfo_list.append(brand)
    prdInfo_list.append(reg_date)
    prdInfo_list.append(num_like)
    prdInfo_list.append(price)
    
    return prdInfo_list
    

In [8]:
def get_prdRvw(url):
    """
    Args:
        url(string): 각 상품의 상세pager
    Returns:
        reviews_list(list): 제조사, 브랜드명, 등록일, 별점, 리뷰 제목, 리뷰 본문, 이미지url
    """
    
    options = webdriver.ChromeOptions()	
    #options.add_argument('headless')	
    options.add_argument('disable-gpu')	
    driver = webdriver.Chrome('chromedriver.exe', options=options)
    driver.get(url)
    time.sleep(3)
    test = driver.find_elements_by_tag_name('#_review_list > li')

    review_list = list()

    # 첫페이지임을 나타내는 매개 변수 first
    first = True

    if first == True:
        first = False
        print('1 ~ 10page')
        page = 2

        while (page <= 11):

            test = driver.find_elements_by_tag_name('#_review_list > li')

            for cnt in test:
                temp_review_list = list()

                score = cnt.text.split("\n")[1].split(' ')[0]
                brand = cnt.text.split("\n")[1].split(' ')[1]
                cus_id = cnt.text.split("\n")[1].split(' ')[2]
                try:
                    review_date = str(20)+cnt.text.split("\n")[1].split(' ')[3]
                except Exception as e:
                    print(e)
                    print(str(cnt.text.split("\n")[1].split(' ')))
                    review_date = str(cnt.text.split("\n")[1].split(' '))
                review_title = cnt.find_element_by_tag_name('div > p').text
                review_text = cnt.find_element_by_tag_name('div > div.atc').text

                # 썸네일
                # 1. 사진
                try: 
                    thumb = cnt.find_element_by_tag_name('div.thmb > span > img').get_attribute('src')
                # 2. 영상
                except:
                    try:
                        thumb = cnt.find_element_by_tag_name('div.thmb > button > img').get_attribute('src')
                # 3. 없는 경우
                    except:
                        thumb = "NULL"

                test = driver.find_elements_by_tag_name('#_review_list > li')        
                temp_review_list.append(score)
                temp_review_list.append(brand)
                temp_review_list.append(cus_id)
                temp_review_list.append(review_date)
                temp_review_list.append(review_title)
                temp_review_list.append(review_text)
                temp_review_list.append(thumb)
                review_list.append(temp_review_list)

            try:
                print('#_review_paging > a:nth-child(%s)완료' %str(page))
                page += 1
                next_page = driver.find_element_by_tag_name('#_review_paging > a:nth-child(%s)' %str(page))
                next_page.click()
            except:
                print("리뷰 마지막 페이지")
                print('#_review_paging > a:nth-child(%s)' %str(page))
                break
            time.sleep(5)



    if first == False:
        print('11page ~ ')
        page = 4

        while (page <= 13):
            test = driver.find_elements_by_tag_name('#_review_list > li')
            for cnt in test:
                temp_review_list = list()

                score = cnt.text.split("\n")[1].split(' ')[0]
                brand = cnt.text.split("\n")[1].split(' ')[1]
                cus_id = cnt.text.split("\n")[1].split(' ')[2]
                try:
                    review_date = str(20)+cnt.text.split("\n")[1].split(' ')[3]
                except Exception as e:
                    print(e)
                    print(str(cnt.text.split("\n")[1].split(' ')))
                    review_date = str(cnt.text.split("\n")[1].split(' '))
                review_title = cnt.find_element_by_tag_name('div > p').text
                review_text = cnt.find_element_by_tag_name('div > div.atc').text

                # 썸네일
                # 1. 사진
                try: 
                    thumb = cnt.find_element_by_tag_name('div.thmb > span > img').get_attribute('src')
                # 2. 영상
                except:
                    try:
                        thumb = cnt.find_element_by_tag_name('div.thmb > button > img').get_attribute('src')
                # 3. 없는 경우
                    except:
                        thumb = "NULL"

                test = driver.find_elements_by_tag_name('#_review_list > li')        
                temp_review_list.append(score)
                temp_review_list.append(brand)
                temp_review_list.append(cus_id)
                temp_review_list.append(review_date)
                temp_review_list.append(review_title)
                temp_review_list.append(review_text)
                temp_review_list.append(thumb)
                review_list.append(temp_review_list)

            try:
                print('#_review_paging > a:nth-child(%s) 완료' %str(page))
                page += 1
                if page == 14:
                    page = 4
                    print('10page 지남')
                next_page = driver.find_element_by_tag_name('#_review_paging > a:nth-child(%s)' %str(page))
                next_page.click()
            except:
                print("리뷰 마지막 페이지")
                print('#_review_paging > a:nth-child(%s)' %str(page))
                break
            time.sleep(5)

    return review_list
    

In [9]:
def write_product_csv(keyword, product_name, product_info_list):
    """ (string, string, list) -> None
    Args:
        filename(string): 검색 키워드
        product_name(string): 제품 이름
        product_info_list(list): 제품 정보
    Retrun:
        None
        csv파일에 제품 정보 작성
    info_list(list): 상품이름, 가격, 제조사, 브랜드명, 등록일, 찜(좋아요) 갯수,
    """
    filename = keyword + '_' + product_name + '.csv'
    csv_file = open(filename, "w", encoding = 'utf-8-sig', newline= '')
    wr = csv.writer(csv_file)
    wr.writerow(['상품 이름', '가격', '제조사', '브랜드명', '등록일', '찜(좋아요) 갯수'])
    wr.writerow(product_info_list)
    csv_file.close()

In [10]:
def write_review_csv(keyword, product_name, review_lists):
    """ (string, list) -> None
    Args:
        filename(string): 파일 이름
        product_name(string): 제품 이름
        review_list(list in list): 리뷰 정보
        # reviews_lists(list): 상품 이름, 가격, 제조사, 브랜드명, 등록일, 별점, 리뷰 제목, 리뷰 본문, 이미지url
    Retrun:
        None
        csv파일에 리뷰 리스트 t 작성
    """
    filename = keyword + '_' + product_name + '_review.csv'
    csv_file = open(filename, "w", encoding = 'utf-8-sig', newline= '')
    wr = csv.writer(csv_file)
    wr.writerow(['평점', '판매점', '사용자id', '등록일', '리뷰 제목', '리뷰 본문', '이미지url'])
    
    for review in review_lists:
        wr.writerow(review)
    csv_file.close()

In [21]:
def main():
    keyword = '베이비뵨 베이비 캐리어'
    url_list = ['https://search.shopping.naver.com/detail/detail.nhn?nvMid=13603101729&query=%EC%9C%A1%EC%95%84%EC%9A%A9%ED%92%88&NaPm=ct%3Dkd59mc8g%7Cci%3D30b2f5d2411549d4f2a914ad743e03ec4de147ea%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3D29af441112b6ebd490d898185fc625c7daab3b3e']


    # 상품 정보
    prdInfo_list = list()
    # 리뷰 정보
    rvw_list = list()

    filename = keyword + '_' + str(date.today()) + '.csv'
    product_csv_file = open(filename, "a", encoding = 'utf-8-sig', newline= '')
    wr = csv.writer(product_csv_file)
    wr.writerow(['상품 이름', '평점', '제조사', '브랜드명', '등록일', '찜(좋아요) 갯수', '가격'])

    for url in url_list:
        temp_prdInfo_list = get_prdInfo(url)
        prdInfo_list.append(temp_prdInfo_list)

        temp_rvw_list = get_prdRvw(url)
        rvw_list.append(temp_rvw_list)
        print(url, '하나의 상품 정보 및 이에 대한 리뷰 수집 완료')
    print('전체 상품 정보 및 이에 대한 리뷰 수집 완료')

    # 상품정보 파일 작성 (list)
    for prdInfo in prdInfo_list:
        wr.writerow(prdInfo)
    print('상품정보 파일 저장완료')
    product_csv_file.close()


    # 리뷰정보 파일 작성
    for idx, reviews in enumerate(rvw_list):
        filename = keyword + '_' + str(prdInfo_list[idx][0]) + str(date.today()) + '.csv'
        review_csv_file = open(filename, "a", encoding = 'utf-8-sig', newline= '')
        wr = csv.writer(review_csv_file)
        wr.writerow(['평점', '브랜드명', '고객 ID', '등록일', '제목', '본문', '이미지 url'])    

        for review in reviews:
            wr.writerow(review)
        review_csv_file.close()
        print(prdInfo_list[idx][0], '리뷰정보 파일 저장완료')

In [22]:
%%time
if __name__ == "__main__":
    main()

1 ~ 10page
#_review_paging > a:nth-child(2)완료
#_review_paging > a:nth-child(3)완료
#_review_paging > a:nth-child(4)완료
#_review_paging > a:nth-child(5)완료
#_review_paging > a:nth-child(6)완료
#_review_paging > a:nth-child(7)완료
#_review_paging > a:nth-child(8)완료
#_review_paging > a:nth-child(9)완료
#_review_paging > a:nth-child(10)완료
#_review_paging > a:nth-child(11)완료
11page ~ 
#_review_paging > a:nth-child(4) 완료
#_review_paging > a:nth-child(5) 완료
#_review_paging > a:nth-child(6) 완료
#_review_paging > a:nth-child(7) 완료
#_review_paging > a:nth-child(8) 완료
#_review_paging > a:nth-child(9) 완료
#_review_paging > a:nth-child(10) 완료
#_review_paging > a:nth-child(11) 완료
#_review_paging > a:nth-child(12) 완료
#_review_paging > a:nth-child(13) 완료
10page 지남
#_review_paging > a:nth-child(4) 완료
#_review_paging > a:nth-child(5) 완료
#_review_paging > a:nth-child(6) 완료
#_review_paging > a:nth-child(7) 완료
#_review_paging > a:nth-child(8) 완료
#_review_paging > a:nth-child(9) 완료
#_review_paging > a:nth-child(10) 완료


In [33]:
df = pd.read_csv('베이비뵨 베이비 캐리어_베이비뵨2018베이비캐리어원에어2020-07-28.csv', encoding = 'utf-8-sig', engine = 'python')

In [34]:
pos_review = 
neu_review = 
neg_review = 

,평점,브랜드명,고객 ID,등록일,제목,본문,이미지 url
0,5,베이비뵨,gkfl****,2019.07.01.,기존에 타브랜드 아기띠를 구매했어서 그 제품을 사용중이였는데 허리밸트부분이 골반을 ...,기존에 타브랜드 아기띠를 구매했어서 그 제품을 사용중이였는데 허리밸트부분이 골반을 ...,https://ssl.pstatic.net/imgshopping/static/m3/...
1,5,베이비뵨,jsr0****,2019.09.17.,다음달 출산 예정인 만삭 임산부입니다. 원래 베이비페어에 오면 직접 보고 사려고 했...,다음달 출산 예정인 만삭 임산부입니다. 원래 베이비페어에 오면 직접 보고 사려고 했...,https://ssl.pstatic.net/imgshopping/static/m3/...
2,5,베이비뵨,myks****,2020.05.09.,아기띠 고르고 고르다가 베이비뵨으로 선택했습니다 신생아부터 사용할 수 있다고 해서 ...,아기띠 고르고 고르다가 베이비뵨으로 선택했습니다 신생아부터 사용할 수 있다고 해서 ...,https://ssl.pstatic.net/imgshopping/static/m3/...
3,5,롯데닷컴,ani*****,2020.04.03.,아기띠 사려고 정말 브랜드별로 비교해가며한달넘게...,아기띠 사려고 정말 브랜드별로 비교해가며\n한달넘게 고민을 했습니다\n그러다 최종 ...,https://ssl.pstatic.net/imgshopping/static/m3/...
4,5,베이비뵨,ican****,2020.03.25.,신생아용 백*정아기띠를 쓰다가 아기가 처음부터 많이 답답해하기도 했고 7키로가 넘어...,신생아용 백*정아기띠를 쓰다가 아기가 처음부터 많이 답답해하기도 했고 7키로가 넘어...,https://ssl.pstatic.net/imgshopping/static/m3/...
5,5,베이비뵨,pcn9****,2020.04.16.,아기띠 처음 구매하는거라 어떤걸 사야하나 고민이 많았는데 리뷰도 좋고 엄마들 입소문...,아기띠 처음 구매하는거라 어떤걸 사야하나 고민이 많았는데 리뷰도 좋고 엄마들 입소문...,https://ssl.pstatic.net/imgshopping/static/m3/...
6,5,롯데닷컴,gre*****,2020.03.26.,한달전에 샀는데 처음에는 애기가 거부하다가 요즘 잘...,한달전에 샀는데 처음에는 애기가 거부하다가 요즘 잘 앉아있어요. 잠투정 심한데 안아...,https://ssl.pstatic.net/imgshopping/static/m3/...
7,5,베이비뵨,ridg****,2020.07.09.,"베이비뵨 미니 선물받아서 사용했구요, 이번에 친구 출산선물로 구매했어요~매장에 직접...","베이비뵨 미니 선물받아서 사용했구요, 이번에 친구 출산선물로 구매했어요~\n매장에 ...",https://ssl.pstatic.net/imgshopping/static/m3/...
8,5,베이비뵨,sona****,2019.07.17.,"막달에 출산 준비 중인데, 신생아 때부터 사용할 수 있고 평들이 좋아서 구매했어요....","막달에 출산 준비 중인데, 신생아 때부터 사용할 수 있고 평들이 좋아서 구매했어요....",https://ssl.pstatic.net/imgshopping/static/m3/...
9,5,베이비뵨,gree****,2019.07.20.,곧 아기가 태어날 예정이라 아기띠를 고르던중 에**이랑 포** 중에서 고민하고 있었...,곧 아기가 태어날 예정이라 아기띠를 고르던중 에**이랑 포** 중에서 고민하고 있었...,https://ssl.pstatic.net/imgshopping/static/m3/...
